In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Aug 16 17:10:53 2020

@author: wanxiang.shen@u.nus.edu
"""

import warnings, os
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score
from sklearn.metrics import auc as calculate_auc

import matplotlib.pyplot as plt
import seaborn as sns

from aggmap import AggMap, AggModel, loadmap
from aggmap import show
np.random.seed(666) #just for reaptable results

def prc_auc_score(y_true, y_score):
    precision, recall, threshold  = precision_recall_curve(y_true, y_score) #PRC_AUC
    auc = calculate_auc(recall, precision)
    return auc

def score(dfr):
    y_true = dfr.y_true
    y_score = dfr.y_score
    y_pred = dfr.y_score.round()

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    acc = (tp + tn) / sum([tn, fp, fn, tp])
    auc = roc_auc_score(y_true, y_score)

    print('acc: %.3f, roc-auc: %.3f' % (acc,auc))

In [2]:
label = pd.read_excel('./subject_info.xlsx')
data = pd.read_excel('./data_species.xlsx', index_col = 0)

In [3]:
label.groupby('Group').size()

Group
HS               30
Healthy         127
MP               40
Stage_0          27
Stage_III_IV     54
Stage_I_II       69
dtype: int64

## Model-1: Healthy vs. Stage_III_IV:

In [34]:
stage = 'Stage_III_IV'
dfy = label[label.Group.isin(['Healthy', stage])].set_index('Subject_ID')[['Group']] #, 'Stage_I_II', 'Stage_III_IV'
idx=dfy[dfy['Group'] == stage].index
s = data.loc[idx].mean()
flist = s[s > 1e-7].index # the same as the paper used
data1 = data[flist]
data1.shape

(347, 4390)

In [35]:
Y = pd.get_dummies(dfy['Group']).values
dfx = data1.loc[dfy.index]
dfx = np.log(dfx + 1e-8)

In [36]:
dfy.shape, dfx.shape

((181, 1), (181, 4390))

In [37]:
mp = AggMap(dfx, metric = 'correlation')
mp.save('./Stage_III_IV.mp')
mp = loadmap('./Stage_III_IV.mp')

2020-12-13 09:58:41,527 - INFO - [bidd-aggmap] - Calculating distance ...
2020-12-13 09:58:42,752 - INFO - [bidd-aggmap] - the number of process is 16


100%|##########| 4390/4390 [00:06<00:00, 668.47it/s]


In [38]:
mp.fit(cluster_channels = 5, verbose = 0, var_thr = 0)

2020-12-13 10:04:01,443 - INFO - [bidd-aggmap] - applying hierarchical clustering to obtain group information ...
2020-12-13 10:04:14,358 - INFO - [bidd-aggmap] - Applying grid feature map(assignment), this may take several minutes(1~30 min)
2020-12-13 10:05:14,789 - INFO - [bidd-aggmap] - Finished


In [39]:
mp.fmap_shape

(67, 66)

In [40]:
mp.plot_scatter()

2020-12-13 10:05:14,804 - INFO - [bidd-aggmap] - generate file: ./feature points_4390_correlation_umap_scatter
2020-12-13 10:05:14,879 - INFO - [bidd-aggmap] - save html file to ./feature points_4390_correlation_umap_scatter


In [41]:
X = mp.batch_transform(dfx.values, scale_method = 'standard')

100%|##########| 181/181 [00:19<00:00,  9.35it/s]


In [42]:
class_ = pd.get_dummies(dfy['Group']).columns
class_

Index(['Healthy', 'Stage_III_IV'], dtype='object')

In [43]:
outer_fold = 10
repeat_seeds = [8, 16, 32, 64, 128]#5 repeats random seeds 8, 16, 32, 64, 128

run_all_res = []
for repeat_seed in repeat_seeds: 
    
    outer = StratifiedKFold(n_splits = outer_fold, shuffle = True, random_state = repeat_seed)
    outer_idx = outer.split(X, Y[:,0])

    
    run_one_res = []
    for i, idx in enumerate(outer_idx):
        
        fold_num = "fold_%s" % str(i).zfill(2) 
        
        train_idx, test_idx = idx
        
        testY = Y[test_idx]
        testX = X[test_idx]
        
        trainX = X[train_idx]
        trainY = Y[train_idx]

        print("\n input train and test X shape is %s, %s " % (trainX.shape,  testX.shape))
        
        clf = AggModel.MultiClassEstimator(epochs = 50, gpuid = 3, conv1_kernel_size = 3,
                                           batch_size = 2, verbose = 0) #
        clf.fit(trainX, trainY ) 

        ## save model
        clf._model.save(os.path.join('./Stage_III_IV_model', 'seed_%s_fold_%s.h5' % (repeat_seed, i)))
    
        y_true = testY[:,1] #Stage
        y_score = clf.predict_proba(testX)[:,1]
        
        dfr = pd.DataFrame([y_true, y_score]).T
        dfr.columns = ['y_true', 'y_score']
        dfr.index = dfy.iloc[test_idx].index
        
        score(dfr)
        
        run_one_res.append(dfr)

    run_all_res.append(pd.concat(run_one_res))


 input train and test X shape is (162, 67, 66, 5), (19, 67, 66, 5) 
MultiClassEstimator(batch_size=2, conv1_kernel_size=3, epochs=50, gpuid='3')
acc: 0.737, roc-auc: 0.821

 input train and test X shape is (163, 67, 66, 5), (18, 67, 66, 5) 
MultiClassEstimator(batch_size=2, conv1_kernel_size=3, epochs=50, gpuid='3')
acc: 0.889, roc-auc: 0.846

 input train and test X shape is (163, 67, 66, 5), (18, 67, 66, 5) 
MultiClassEstimator(batch_size=2, conv1_kernel_size=3, epochs=50, gpuid='3')
acc: 0.833, roc-auc: 0.831

 input train and test X shape is (163, 67, 66, 5), (18, 67, 66, 5) 
MultiClassEstimator(batch_size=2, conv1_kernel_size=3, epochs=50, gpuid='3')
acc: 0.944, roc-auc: 1.000

 input train and test X shape is (163, 67, 66, 5), (18, 67, 66, 5) 
MultiClassEstimator(batch_size=2, conv1_kernel_size=3, epochs=50, gpuid='3')
acc: 0.944, roc-auc: 1.000

 input train and test X shape is (163, 67, 66, 5), (18, 67, 66, 5) 
MultiClassEstimator(batch_size=2, conv1_kernel_size=3, epochs=50, 

In [44]:
results = []
for dfr, repeat_seed in zip(run_all_res,repeat_seeds):

    y_true = dfr.y_true
    y_score = dfr.y_score
    y_pred = dfr.y_score.round()


    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    acc = (tp + tn) / sum([tn, fp, fn, tp])

    sensitivity = tp / sum([tp, fn])
    specificity = tn / sum([tn, fp])

    prc_auc = prc_auc_score(y_true, y_score)
    roc_auc = roc_auc_score(y_true, y_score)

    precision = tp / sum([tp, fp])
    recall =  tp / sum([tp, fn]) #equals to sensitivity

    res = {
           'repeat_seed':repeat_seed,

           'accuracy':acc, 

           'prc_auc':prc_auc, 
           'roc_auc':roc_auc,

           'sensitivity': sensitivity, 
           'specificity': specificity,

           'precision':precision,
           'recall':recall,
           'F1': 2*precision*sensitivity/(precision+sensitivity)
          }
    
    results.append(res)

In [45]:
pd.DataFrame(results).roc_auc.mean()

0.9098571011956839

In [47]:
pd.DataFrame(results)

,repeat_seed,accuracy,prc_auc,roc_auc,sensitivity,specificity,precision,recall,F1
0,8,0.872928,0.836373,0.910032,0.629630,0.976378,0.918919,0.629630,0.747253
1,16,0.883978,0.850824,0.916448,0.666667,0.976378,0.923077,0.666667,0.774194
2,32,0.867403,0.823759,0.905949,0.648148,0.960630,0.875000,0.648148,0.744681
3,64,0.850829,0.844937,0.911490,0.592593,0.960630,0.864865,0.592593,0.703297
4,128,0.850829,0.819973,0.905366,0.574074,0.968504,0.885714,0.574074,0.696629
